<a href="https://colab.research.google.com/github/anindita/dances/blob/main/analyse/segmentation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation 1

This has been modified by code written by [tugstugi](https://github.com/tugstugi/dl-colab-notebooks)

# Make images from video

In [ ]:
YOUTUBE_ID = 'Y4ARBzok9aU'

!pip install -q youtube-dl
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:05:28 -t 2 video.mp4

In [ ]:
!mkdir images

video_size=(320, 240)

In [ ]:

import cv2
import os
import numpy as np
  

cam = cv2.VideoCapture("video.mp4")
currentframe = 0
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
  
while(True):
    ret,frame = cam.read()
  
    if ret:
        name = '/content/images/' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        frame = cv2.resize(frame, (1280,720))
        frame = frame[0:720, 160:1120]
        frame = cv2.resize(frame, video_size)
        frame = cv2.filter2D(frame, -1, kernel)
        cv2.imwrite(name, frame)

        currentframe += 1
    else:
        break
  
cam.release()
cv2.destroyAllWindows()

# Try to segment diver from image

In [ ]:
import os
from os.path import exists, join, basename, splitext

import random
import PIL
import torchvision
import cv2
import numpy as np
import torch
torch.set_grad_enabled(False)
  
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
model = model.eval().cuda()

In [ ]:

def segment(image_file):
  image = PIL.Image.open(image_file)
  image_tensor = torchvision.transforms.functional.to_tensor(image).cuda()
  output = model(image_tensor.unsqueeze(0))


  # visualization code copied from: https://github.com/fregu856/deeplabv3/blob/master/visualization/run_on_seq.py
  #
  outputs = output['out'].cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
  pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
  pred_label_imgs = pred_label_imgs.astype(np.uint8)

  label_names = ['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'] 
  def label_img_to_color(img):
      label_to_color = {
          0: [128, 64,128], 1: [244, 35,232], 2: [ 70, 70, 70], 3: [102,102,156],
          4: [190,153,153], 5: [153,153,153], 6: [250,170, 30], 7: [220,220,  0],
          8: [107,142, 35], 9: [152,251,152], 10: [ 70,130,180], 11: [220, 20, 60],
          12: [255,  0,  0], 13: [  0,  0,142], 14: [  0,  0, 70], 15: [  0, 60,100],
          16: [  0, 80,100], 17: [  0,  0,230], 18: [119, 11, 32], 19: [81,  0, 81]
          }

      img_height, img_width = img.shape
      img_color = np.zeros((img_height, img_width, 3))
      for row in range(img_height):
          for col in range(img_width):
              label = img[row, col]
              img_color[row, col] = np.array(label_to_color[label])

      return img_color


  i = 0
  pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
  img = image_tensor #imgs[i] # (shape: (3, img_h, img_w))

  img = img.data.cpu().numpy()
  img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
  img = img*np.array([0.229, 0.224, 0.225])
  img = img + np.array([0.485, 0.456, 0.406])
  img = img*255.0
  img = img.astype(np.uint8)

  pred_label_img_color = label_img_to_color(pred_label_img)
  overlayed_img = 0.35*img + 0.65*pred_label_img_color
  overlayed_img = overlayed_img.astype(np.uint8)

  plt.figure(figsize=(10, 8))
  plt.imshow(overlayed_img)

arr = os.listdir('/content/images')
for f in arr:
  segment(f'/content/images/{f}')